loading datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/MLDL_repo/GTA5.zip'
extract_path = '/content/dataset'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/MLDL_repo/Cityscapes.zip'
extract_path = '/content/cityscapes'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Extraction complete!


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/MLDL_repo/step3b')

In [ ]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=657452c6cc1dab38d43734106e5860923d975d8f167beebf99416aa4bf955804
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=86587c5421eb6560a4af3c61db80af8cb40da6f453cc44c3d635c5cffee95544
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath


find checkpoint function

In [ ]:
import re
import os
import re

def find_latest_checkpoint(checkpoint_dir):
    if not os.path.exists(checkpoint_dir):
        print(f"Checkpoint directory {checkpoint_dir} does not exist.")
        return None

    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("bisenet_epoch_") and f.endswith(".pt")]
    if not checkpoints:
        print(f"No checkpoints found in {checkpoint_dir}.")
        return None

    def extract_epoch(fname):
        match = re.search(r"bisenet_epoch_(\d+).pt", fname)
        return int(match.group(1)) if match else -1

    checkpoints.sort(key=extract_epoch, reverse=True)

    latest = os.path.join(checkpoint_dir, checkpoints[0])
    print(f" Found latest checkpoint: {latest}")
    return latest


data augmentations trasformation

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import transforms

def get_train_transform():
    return A.Compose([
        A.GaussNoise(var_limit=(1.0, 5.0), p=0.5),
        A.Resize(720, 1280),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])

main

In [ ]:
import os
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from torchvision import transforms as T
from PIL import Image
from datasets_custom.gta5_aug import GTA5
from models.bisenet.build_bisenet import BiSeNet
from train import train_one_epoch
from train import validate

def main():

    dataset_root = '/content/dataset/GTA5'

    # Create the full dataset without transforms first
    full_dataset = GTA5(root=dataset_root, transform=None)

    # Then split
    indices = list(range(len(full_dataset)))
    train_indices, val_indices = train_test_split(indices, test_size=0.30, random_state=42)

    # Create two GTA5 datasets with different transforms
    train_dataset = GTA5(root=dataset_root, transform=get_train_transform())

    # Subset the datasets
    train_dataset = Subset(train_dataset, train_indices)
    print(f"Train dataset size: {len(train_dataset)}")


    train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    num_classes = 19
    base_lr = 2.5e-4
    batch_size = 2
    epochs = 50
    context_path = 'resnet18'
    checkpoint_dir = "/content/drive/MyDrive/checkpoints_3b_GaussianNoise"

    if not os.path.exists(checkpoint_dir):
      os.makedirs(checkpoint_dir)

    model = BiSeNet(num_classes=num_classes, context_path=context_path)
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs")
        model = nn.DataParallel(model)

    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    start_epoch = 0
    start_batch = 0

    latest_ckpt = find_latest_checkpoint(checkpoint_dir)
    print("latest_ckpt is:", latest_ckpt)

    if latest_ckpt:
        print(f"Restore from checkpoint: {latest_ckpt}")
        checkpoint = torch.load(latest_ckpt, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        start_batch = 0
        print(f"Picking up from epoch {start_epoch}")
    else:
        print("No checkpoint found, start training from scratch")

    # Check Data and Labels
    for images, labels in train_loader:
        print("Images shape:", images.shape, "dtype:", images.dtype)
        print("Labels shape:", labels.shape, "dtype:", labels.dtype)
        print("Unique labels:", torch.unique(labels))
        break  # Print for the first batch only

    # Training
    for epoch in range(start_epoch, epochs):
        current_start_batch = start_batch if epoch == start_epoch else 0
        train_one_epoch(model, train_loader, optimizer, base_lr, epoch, epochs, device,
                        checkpoint_dir=checkpoint_dir, start_batch=current_start_batch)

    final_model_path = "/content/drive/MyDrive/MLDL_repo/step3b/final_models/bisenet_final_3b_GaussianNoise.pht"
    torch.save(model.state_dict(), final_model_path)
    print(f"Modello finale salvato in: {final_model_path}")


if __name__ == "__main__":
    main()


Train dataset size: 1750


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 150MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 149MB/s]


The latest checkpoint is: bisenet_epoch_49.pt
latest_ckpt is: /content/drive/MyDrive/checkpoints_3b_GaussianNoise/bisenet_epoch_49.pt
Restore from checkpoint: /content/drive/MyDrive/checkpoints_3b_GaussianNoise/bisenet_epoch_49.pt
Picking up from epoch 50
Images shape: torch.Size([2, 3, 720, 1280]) dtype: torch.float32
Labels shape: torch.Size([2, 720, 1280]) dtype: torch.uint8
Unique labels: tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  13,  14,
         15,  16, 255], dtype=torch.uint8)
Modello finale salvato in: /content/drive/MyDrive/checkpoints_3b_GaussianNoise/bisenet_final_3b_GaussianNoise.pht


In [ ]:
import os
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from torchvision import transforms as T
from PIL import Image
from datasets_custom.gta5_aug import GTA5
from datasets_custom.cityscapes import CityScapes
from models.bisenet.build_bisenet import BiSeNet
from datasets_custom.labels import GTA5Labels_TaskCV2017
from train import validate


#initialize model
num_classes = 19
context_path = 'resnet18'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BiSeNet(num_classes=num_classes, context_path=context_path).to(device)


# load model
model_path = r"/content/drive/MyDrive/MLDL_repo/step3b/final_models/bisenet_final_3b_GaussianNoise.pht"
model.load_state_dict(torch.load(model_path, map_location='cpu'), strict = False )

model.eval() #python built in function

# Dataset definitions and transformation
transform = transforms.Compose([
        transforms.Resize((512, 1024)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
    ])

target_transform = transforms.Compose([
        transforms.Resize((512, 1024), interpolation=transforms.InterpolationMode.NEAREST),
        transforms.PILToTensor()
    ])

dataset_root = '/content/cityscapes/Cityscapes/Cityspaces'

test_dataset = CityScapes(root=dataset_root, split='val', transform=transform, target_transform=target_transform)
print(f"Test dataset size: {len(test_dataset)}")
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True, num_workers=2)


# results
best_miou, miou, per_class_ious = validate(model, test_loader, num_classes, device, best_miou=0.0)
print("\n Validation Results on Cityscape:")
print(f" - mIoU: {miou:.4f}")
for idx, label in enumerate(GTA5Labels_TaskCV2017.list_):
    print(f"{label.name:>15}: IoU = {per_class_ious[idx]:.4f}")

Loaded 500 images for split: val
Test dataset size: 500


Validating: 100%|██████████| 250/250 [00:57<00:00,  4.37it/s]

 Validation mIoU: 0.1603
 New best mIoU found!

 Validation Results on Cityscape:
 - mIoU: 0.1603
           road: IoU = 0.1768
       sidewalk: IoU = 0.0870
       building: IoU = 0.5970
           wall: IoU = 0.0193
          fence: IoU = 0.0059
           pole: IoU = 0.0000
          light: IoU = 0.0072
           sign: IoU = 0.0046
     vegetation: IoU = 0.6596
        terrain: IoU = 0.0297
            sky: IoU = 0.7163
         person: IoU = 0.2264
          rider: IoU = 0.0007
            car: IoU = 0.4177
          truck: IoU = 0.0348
            bus: IoU = 0.0080
          train: IoU = 0.0000
      motocycle: IoU = 0.0529
        bicycle: IoU = 0.0010
